In [ ]:
#importing modules
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from matplotlib import pyplot
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import datetime
import time
import os

In [ ]:
#loading train dataset
df_full = pd.read_csv(#test_data_url_here)
print(df_full.shape)
#df_full.head()

In [ ]:
#making dummies for interaction effects
df_full['fem_1'] = 0
df_full.loc[(df_full['female']==1)&(df_full['pclass_1']==1), 'fem_1'] = 1

df_full['fem_2'] = 0
df_full.loc[(df_full['female']==1)&(df_full['pclass_2']==1), 'fem_2'] = 1

df_full['fem_3'] = 0
df_full.loc[(df_full['female']==1)&(df_full['pclass_3']==1), 'fem_3'] = 1

df_full['male_1'] = 0
df_full.loc[(df_full['male']==1)&(df_full['pclass_1']==1), 'male_1'] = 1

df_full['male_2'] = 0
df_full.loc[(df_full['male']==1)&(df_full['pclass_2']==1), 'male_2'] = 1

df_full['male_3'] = 0
df_full.loc[(df_full['male']==1)&(df_full['pclass_3']==1), 'male_3'] = 1

In [ ]:
#breaking down into X and y arrays, removing categorical vars: train
X = df_full.drop(['Survived','Name','Sex','Ticket','Embarked','Pclass'], axis = 1)
y = pd.DataFrame(df_full['Survived'])
print(X.shape)
print(y.shape)
#X.head()

In [ ]:
#train, dev split
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.1)

In [ ]:
#implementing the model
import statsmodels.api as sm
logit_model=sm.Logit(y_train,X_train)
result=logit_model.fit()
print(result.summary2())

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
#predicting and calculating accuracy
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [ ]:
#convert to array
y_dev = y_dev['Survived'].to_numpy()
y_dev

In [ ]:
#Confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
sns.heatmap(confusion_matrix, annot=True)

In [ ]:
#calculating accuracy and other scores
accuracy_dev = metrics.accuracy_score(y_dev, y_pred)
print('accuracy is: '+ str(accuracy_dev))
precision_dev = metrics.precision_score(y_dev, y_pred)
print(precision_dev)
sensitivity_dev = metrics.recall_score(y_dev, y_pred)
print(sensitivity_dev)
f1_score_dev = metrics.f1_score(y_dev, y_pred)
print(f1_score_dev)

In [ ]:
#loading test dataset
X_test = pd.read_csv(r"test_datasets/test_clean_full.csv")
print(X_test.shape)
X_test.head()

In [ ]:
#making dummies for interaction effects
X_test['fem_1'] = 0
X_test.loc[(X_test['female']==1)&(X_test['pclass_1']==1), 'fem_1'] = 1

X_test['fem_2'] = 0
X_test.loc[(X_test['female']==1)&(X_test['pclass_2']==1), 'fem_2'] = 1

X_test['fem_3'] = 0
X_test.loc[(X_test['female']==1)&(X_test['pclass_3']==1), 'fem_3'] = 1

X_test['male_1'] = 0
X_test.loc[(X_test['male']==1)&(X_test['pclass_1']==1), 'male_1'] = 1

X_test['male_2'] = 0
X_test.loc[(X_test['male']==1)&(X_test['pclass_2']==1), 'male_2'] = 1

X_test['male_3'] = 0
X_test.loc[(X_test['male']==1)&(X_test['pclass_3']==1), 'male_3'] = 1

In [ ]:
#breaking down into X and y arrays, removing categorical vars - these vars(except Name) have already been made into booleans: train
X_test = X_test.drop(['Name','Sex','Ticket','Embarked','Pclass','male','Parch', 'SibSp','PassengerId','interaction_QC'], axis = 1)
print(X_test.shape)
print(X.shape)
#X.head()

In [ ]:
#prediction y for dev data 
y_pred_test=logreg.predict(X_test)

In [ ]:
#reading a submission csv and then using it's passenger id
pid_array = pd.read_csv(r'submission\csv_submission20210529230942.csv')
pid_array.shape
#pid_array.head()

In [ ]:
#subsetting only the Passenger Id column for final submission 
X_concat = pid_array[['PassengerId']]
X_concat.head()

In [ ]:
#creating final submission csv
y_pred_test_df = pd.DataFrame(y_pred_test)
y_pred_test_df.rename(columns = {0:'Survived'},inplace = True)
csv_submission = pd.merge(X_concat,y_pred_test_df, left_index=True, right_index=True)
print(csv_submission.shape)
csv_submission.head()

In [ ]:
#saving csv with a unique timestamp
timestamp = str(datetime.datetime.now().strftime('%Y%m%d%H%M%S'))
timestamp
csv_submission.to_csv(r'submission\csv_submission' + timestamp + '.csv', index =False)